# Regression : Prediction des prix immobiliers

> Ce notebook est un exemple pratique clef en main pour apprendre les techniques de regression en Machine Learning.
> Nous allons predire le prix de biens immobiliers a partir de leurs caracteristiques (surface, nombre de pieces, ville, etc.).
>
> **Objectifs :**
> - Comprendre le pipeline complet d'un projet de regression
> - Comparer plusieurs algorithmes (Regression lineaire, Ridge, Lasso, Random Forest)
> - Evaluer les performances avec les metriques adaptees (MSE, RMSE, MAE, R2, MAPE)
> - Analyser les residus pour valider le modele

In [ ]:
# ============================================================
# Cellule 1 : Imports et chargement des donnees
# ============================================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, learning_curve
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    r2_score,
    mean_absolute_percentage_error,
)

import warnings
warnings.filterwarnings("ignore")

# Configuration graphique
plt.rcParams["figure.figsize"] = (10, 6)
plt.rcParams["figure.dpi"] = 100
sns.set_style("whitegrid")

# Chargement des donnees
df = pd.read_csv("../data/house_prices.csv")

print(f"Dataset charge avec succes : {df.shape[0]} lignes, {df.shape[1]} colonnes")
df.head(10)

In [ ]:
# ============================================================
# Cellule 2 : Analyse exploratoire (EDA)
# ============================================================

print("=" * 60)
print("INFORMATIONS GENERALES SUR LE DATASET")
print("=" * 60)
print(f"\nDimensions : {df.shape}")
print(f"\nTypes des colonnes :")
print(df.dtypes)

print("\n" + "=" * 60)
print("VALEURS MANQUANTES")
print("=" * 60)
print(df.isnull().sum())

print("\n" + "=" * 60)
print("STATISTIQUES DESCRIPTIVES (VARIABLES NUMERIQUES)")
print("=" * 60)
df.describe().round(2)

## Visualisations exploratoires

Nous allons examiner les relations entre les variables explicatives et la variable cible (prix).

In [ ]:
# ============================================================
# Cellule 3 : Visualisations
# ============================================================

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Scatter plot : prix vs surface
axes[0, 0].scatter(df["surface_m2"], df["prix"], alpha=0.7, edgecolors="k", linewidth=0.5)
axes[0, 0].set_xlabel("Surface (m2)")
axes[0, 0].set_ylabel("Prix (euros)")
axes[0, 0].set_title("Prix en fonction de la surface")

# Scatter plot : prix vs nombre de pieces
axes[0, 1].scatter(df["nb_pieces"], df["prix"], alpha=0.7, edgecolors="k", linewidth=0.5, color="orange")
axes[0, 1].set_xlabel("Nombre de pieces")
axes[0, 1].set_ylabel("Prix (euros)")
axes[0, 1].set_title("Prix en fonction du nombre de pieces")

# Boxplot par ville
ordre_villes = df.groupby("ville")["prix"].median().sort_values(ascending=False).index
sns.boxplot(data=df, x="ville", y="prix", order=ordre_villes, ax=axes[1, 0], palette="Set2")
axes[1, 0].set_xlabel("Ville")
axes[1, 0].set_ylabel("Prix (euros)")
axes[1, 0].set_title("Distribution des prix par ville")
axes[1, 0].tick_params(axis="x", rotation=45)

# Distribution des prix
axes[1, 1].hist(df["prix"], bins=15, edgecolor="black", alpha=0.7, color="steelblue")
axes[1, 1].set_xlabel("Prix (euros)")
axes[1, 1].set_ylabel("Frequence")
axes[1, 1].set_title("Distribution des prix")

plt.tight_layout()
plt.show()

# Matrice de correlation (heatmap)
print("\n--- Matrice de correlation ---")
colonnes_num = ["surface_m2", "nb_pieces", "etage", "parking", "annee_construction", "prix"]
fig, ax = plt.subplots(figsize=(8, 6))
matrice_corr = df[colonnes_num].corr()
sns.heatmap(matrice_corr, annot=True, fmt=".2f", cmap="coolwarm", center=0, ax=ax)
ax.set_title("Matrice de correlation")
plt.tight_layout()
plt.show()

## Preprocessing des donnees

Avant d'entrainer nos modeles, nous devons :
1. **Encoder** la variable categorielle `ville` (Label Encoding ou One-Hot Encoding)
2. **Separer** les donnees en jeu d'entrainement et jeu de test

In [ ]:
# ============================================================
# Cellule 4 : Preprocessing (encodage + split train/test)
# ============================================================

# Copie du dataframe pour ne pas modifier l'original
df_model = df.copy()

# One-Hot Encoding de la variable 'ville'
df_model = pd.get_dummies(df_model, columns=["ville"], drop_first=True)

print("Colonnes apres encodage :")
print(df_model.columns.tolist())

# Separation features / target
X = df_model.drop("prix", axis=1)
y = df_model["prix"]

# Split train/test (80/20)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"\nTaille du jeu d'entrainement : {X_train.shape[0]} echantillons")
print(f"Taille du jeu de test : {X_test.shape[0]} echantillons")
print(f"Nombre de features : {X_train.shape[1]}")

## Modele 1 : Regression lineaire

Nous commencons par le modele le plus simple : la regression lineaire ordinaire (OLS).

## Comprendre les metriques de regression

Avant de lancer notre premier modele, prenons le temps de comprendre **comment on mesure la qualite** d'une prediction. Imaginez que vous etes agent immobilier et que vous estimez des prix de maisons :

### L'analogie du GPS

Pensez a un GPS qui vous dit : "Vous arriverez dans 30 minutes".

- Si vous arrivez en **28 minutes** : l'erreur est de 2 minutes (pas grave)
- Si vous arrivez en **55 minutes** : l'erreur est de 25 minutes (probleme !)

Les metriques de regression mesurent exactement ca : **a quel point nos predictions sont loin de la realite**.

### Les metriques expliquees simplement

| Metrique | En langage courant | Exemple concret |
|----------|-------------------|-----------------|
| **MAE** (Mean Absolute Error) | "En moyenne, je me trompe de X euros" | MAE = 15 000 → le modele se trompe de 15 000€ en moyenne |
| **RMSE** (Root Mean Squared Error) | "Comme la MAE, mais penalise plus les grosses erreurs" | Si une maison est predite a 100 000€ au lieu de 300 000€, le RMSE sera tres eleve |
| **R²** (R-squared) | "Quelle proportion du prix le modele arrive-t-il a expliquer ?" | R² = 0.85 → le modele explique 85% des variations de prix |
| **MAPE** | "En pourcentage, je me trompe de X%" | MAPE = 8% → les predictions sont a +/- 8% du vrai prix |

### Comment lire les resultats ?

```
Bon modele          Mauvais modele
─────────────       ─────────────
MAE  : 10 000€      MAE  : 80 000€
RMSE : 15 000€      RMSE : 120 000€
R²   : 0.92         R²   : 0.35
MAPE : 5%           MAPE : 40%
```

> **A retenir** : Le R² est le plus intuitif pour debuter. Un R² de 0 = le modele ne predit rien. Un R² de 1 = prediction parfaite. En pratique, un R² > 0.80 est souvent considere comme bon.

In [ ]:
# ============================================================
# Cellule 5 : Regression lineaire + metriques
# ============================================================

def evaluer_modele(modele, X_train, X_test, y_train, y_test, nom_modele):
    """Fonction utilitaire pour evaluer un modele de regression."""
    # Predictions
    y_pred_train = modele.predict(X_train)
    y_pred_test = modele.predict(X_test)

    # Calcul des metriques sur le jeu de test
    mse = mean_squared_error(y_test, y_pred_test)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred_test)
    r2 = r2_score(y_test, y_pred_test)
    mape = mean_absolute_percentage_error(y_test, y_pred_test) * 100

    # R2 sur le train pour detecter le surapprentissage
    r2_train = r2_score(y_train, y_pred_train)

    print(f"\n{'=' * 50}")
    print(f"  RESULTATS : {nom_modele}")
    print(f"{'=' * 50}")
    print(f"  R2 (train) : {r2_train:.4f}")
    print(f"  R2 (test)  : {r2:.4f}")
    print(f"  MSE        : {mse:,.0f}")
    print(f"  RMSE       : {rmse:,.0f} euros")
    print(f"  MAE        : {mae:,.0f} euros")
    print(f"  MAPE       : {mape:.2f}%")

    return {
        "Modele": nom_modele,
        "R2_train": r2_train,
        "R2_test": r2,
        "RMSE": rmse,
        "MAE": mae,
        "MAPE": mape,
    }


# Entrainement du modele de regression lineaire
lr = LinearRegression()
lr.fit(X_train, y_train)

# Evaluation
resultats = []
resultats.append(evaluer_modele(lr, X_train, X_test, y_train, y_test, "Regression Lineaire"))

# Affichage des coefficients
print("\n--- Coefficients du modele lineaire ---")
coefs = pd.DataFrame({
    "Feature": X_train.columns,
    "Coefficient": lr.coef_
}).sort_values("Coefficient", ascending=False)
print(coefs.to_string(index=False))

## Modeles 2 et 3 : Ridge et Lasso

Les regressions Ridge (L2) et Lasso (L1) ajoutent une penalite de regularisation pour eviter le surapprentissage.

- **Ridge** : penalise la somme des carres des coefficients (les reduit mais ne les annule pas)
- **Lasso** : penalise la somme des valeurs absolues des coefficients (peut les annuler, utile pour la selection de features)

In [ ]:
# ============================================================
# Cellule 6 : Ridge et Lasso
# ============================================================

# Regression Ridge
ridge = Ridge(alpha=1.0, random_state=42)
ridge.fit(X_train, y_train)
resultats.append(evaluer_modele(ridge, X_train, X_test, y_train, y_test, "Ridge (alpha=1.0)"))

# Regression Lasso
lasso = Lasso(alpha=100.0, random_state=42)
lasso.fit(X_train, y_train)
resultats.append(evaluer_modele(lasso, X_train, X_test, y_train, y_test, "Lasso (alpha=100.0)"))

# Comparaison des coefficients Ridge vs Lasso
print("\n--- Comparaison des coefficients ---")
comp_coefs = pd.DataFrame({
    "Feature": X_train.columns,
    "Lineaire": lr.coef_,
    "Ridge": ridge.coef_,
    "Lasso": lasso.coef_,
})
print(comp_coefs.round(2).to_string(index=False))

## Modele 4 : Random Forest Regression

Le Random Forest est un modele d'ensemble base sur des arbres de decision.
Il est capable de capturer des relations non lineaires entre les variables.

In [ ]:
# ============================================================
# Cellule 7 : Random Forest Regression
# ============================================================

rf = RandomForestRegressor(
    n_estimators=100,
    max_depth=10,
    random_state=42,
    n_jobs=-1,
)
rf.fit(X_train, y_train)

resultats.append(evaluer_modele(rf, X_train, X_test, y_train, y_test, "Random Forest"))

# Importance des features
print("\n--- Importance des features (Random Forest) ---")
importances = pd.DataFrame({
    "Feature": X_train.columns,
    "Importance": rf.feature_importances_
}).sort_values("Importance", ascending=False)

fig, ax = plt.subplots(figsize=(8, 5))
sns.barplot(data=importances, x="Importance", y="Feature", palette="viridis", ax=ax)
ax.set_title("Importance des features - Random Forest")
ax.set_xlabel("Importance")
ax.set_ylabel("")
plt.tight_layout()
plt.show()

## Comparaison des modeles

Nous allons maintenant comparer les performances de tous les modeles dans un tableau recapitulatif,
puis tracer les courbes d'apprentissage du meilleur modele.

## Analyse des residus : verifier que le modele est fiable

Un **residu**, c'est simplement la difference entre le prix reel et le prix predit par le modele.

> **Exemple** : Si une maison vaut 250 000€ et que le modele predit 240 000€, le residu est de +10 000€ (le modele a sous-estime de 10 000€).

### Pourquoi analyser les residus ?

C'est comme verifier qu'un thermometre est fiable : on ne regarde pas juste s'il donne la bonne temperature en moyenne, on verifie aussi qu'il ne se trompe pas systematiquement dans un sens.

**4 choses a verifier :**

1. **Predictions vs Realite** (graphique en haut a gauche) : les points doivent etre proches de la ligne rouge (= prediction parfaite)
2. **Distribution des residus** (en haut a droite) : doit ressembler a une cloche centree sur 0 (le modele se trompe autant vers le haut que vers le bas)
3. **Residus vs Predictions** (en bas a gauche) : les points doivent etre repartis uniformement autour de 0 (pas de pattern en forme de V ou d'entonnoir)
4. **QQ-Plot** (en bas a droite) : les points doivent suivre la ligne diagonale (confirme la distribution normale des erreurs)

## Conclusion

### Recapitulatif des resultats

Nous avons compare 4 modeles de regression sur notre jeu de donnees immobilier :

| Modele | Avantages | Inconvenients |
|--------|-----------|---------------|
| **Regression lineaire** | Simple, interpretable | Hypothese de linearite |
| **Ridge** | Regularisation L2, robuste | Moins interpretable |
| **Lasso** | Selection de features, parcimonie | Peut eliminer des features utiles |
| **Random Forest** | Non lineaire, robuste | Boite noire, plus lent |

### Points cles

1. La **surface** est le facteur le plus important pour predire le prix
2. La **ville** a un impact significatif (Paris plus cher que les autres villes)
3. Le **Random Forest** capture mieux les relations non lineaires
4. L'analyse des residus confirme la qualite du modele retenu

### Pour aller plus loin

- Tester un **Gradient Boosting** (XGBoost, LightGBM)
- Ajouter des features supplementaires (proximite transports, quartier, etc.)
- Realiser une **validation croisee** plus poussee
- Optimiser les hyperparametres avec **GridSearchCV** ou **RandomizedSearchCV**

### Lexique debutant

| Terme | Definition simple |
|-------|------------------|
| **Regression** | Predire un nombre (prix, temperature, age...) |
| **Feature** | Une information utilisee pour la prediction (surface, ville...) |
| **Train set** | Les donnees sur lesquelles le modele apprend |
| **Test set** | Les donnees reservees pour verifier que le modele generalise |
| **Overfitting** | Le modele a "appris par coeur" le train set et echoue sur de nouvelles donnees |
| **R²** | Score de 0 a 1, mesure la qualite globale du modele (1 = parfait) |
| **Residu** | Difference entre la valeur reelle et la prediction |
| **Regularisation** | Technique pour empecher l'overfitting (Ridge, Lasso) |

In [ ]:
# ============================================================
# Cellule 10 : Resume final
# ============================================================

print("=" * 60)
print("  RESUME FINAL")
print("=" * 60)

# Identification du meilleur modele
meilleur = df_resultats.iloc[0]
print(f"\n  Meilleur modele     : {meilleur['Modele']}")
print(f"  R2 (test)           : {meilleur['R2_test']:.4f}")
print(f"  RMSE                : {meilleur['RMSE']:,.0f} euros")
print(f"  MAE                 : {meilleur['MAE']:,.0f} euros")
print(f"  MAPE                : {meilleur['MAPE']:.2f}%")

print(f"\n  Interpretation : Le modele predit le prix a +/- {meilleur['MAE']:,.0f} euros en moyenne.")
print(f"  Cela represente une erreur relative moyenne de {meilleur['MAPE']:.1f}%.")
print("\n" + "=" * 60)